In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import time
import operator
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Create DataFrames from CSV files
t0 = time.time()
filtered_lyric_SF_df = pd.read_csv('../../Data/filtered_lyric_SF.csv')
filtered_lyric_SF_df = filtered_lyric_SF_df.drop([0,1])
filtered_lyric_SF_df.index = filtered_lyric_SF_df.index - 2
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_lyric_SF_df.head()

Run time: 37.11518383026123 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,cut,ale,lack,slogan,libido,oop,scorch,muslim,heavyweight,mozambiqu
0,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.392,0.574,7.0,1.0,0.1700,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,WITHOUT YOU,The Kid LAROI,pop,8.0,['australian'],0.662,0.413,0.0,1.0,0.0299,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Heat Waves,Glass Animals,pop,8.0,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,1.0,0.0944,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,r u ok,Tate McRae,pop,8.0,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,1.0,0.0373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Create a dictionary of category names and ids
cat_name_id = {}
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    cat_id = list(filtered_lyric_SF_df[filtered_lyric_SF_df['category_name']==cat]['category_id'])[0]
    cat_name_id[cat] = cat_id
    print(cat, cat_id)

blues 0.0
classical 1.0
country 2.0
funk 3.0
hiphop 4.0
indie_alt 5.0
jazz 6.0
metal 7.0
pop 8.0
punk 9.0
rnb 10.0
rock 11.0
romance 12.0
soul 13.0


In [4]:
# Create a genre lists column
t0 = time.time()
genre_lists = []
for index, row in filtered_lyric_SF_df.iterrows():
    genres = row['genre_list'].replace('[', '').replace(']', '').replace("'", '').split(', ')
    genre_lists.append(genres)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(genre_lists)

Run time: 29.753556966781616 seconds


8010

In [5]:
# Replace the genre_list column
filtered_lyric_SF_df['genre_list'] = genre_lists

In [6]:
# Print how many songs there are in each category
for cat in category_list:
    song_count = len(filtered_lyric_SF_df[filtered_lyric_SF_df['category_name']==cat])
    print(f'There are {song_count} songs in the {cat} category.')

There are 423 songs in the blues category.
There are 85 songs in the classical category.
There are 1256 songs in the country category.
There are 280 songs in the funk category.
There are 682 songs in the hiphop category.
There are 654 songs in the indie_alt category.
There are 196 songs in the jazz category.
There are 892 songs in the metal category.
There are 692 songs in the pop category.
There are 569 songs in the punk category.
There are 362 songs in the rnb category.
There are 1228 songs in the rock category.
There are 280 songs in the romance category.
There are 411 songs in the soul category.


In [7]:
# Get the most popular genres for each category
t0 = time.time()
category_genres = {}
for category in category_list:
    genres = []
    for index, row in filtered_lyric_SF_df[filtered_lyric_SF_df['category_name']==category].iterrows():
        genres.extend(row['genre_list'])
    unique_genres = list(set(genres))
    genre_counts = []
    for genre in unique_genres:
        genre_counts.append((genre, genres.count(genre)))
    category_genres[category] = sorted(genre_counts, key = lambda x: x[1], reverse=True)[0:5]
    print(category, category_genres[category])
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

blues [('blues', 260), ('electric', 190), ('modern', 168), ('traditional', 146), ('classic', 106)]
classical [('classical', 13), ('dance', 12), ('operatic', 12), ('', 9), ('pop', 9)]
country [('country', 976), ('contemporary', 847), ('modern', 277), ('pop', 88), ('nashville', 78)]
funk [('funk', 132), ('soul', 117), ('motown', 96), ('quiet', 81), ('disco', 78)]
hiphop [('rap', 345), ('hip', 304), ('pop', 227), ('trap', 192), ('southern', 142)]
indie_alt [('indie', 290), ('alternative', 180), ('new', 153), ('modern', 148), ('art', 141)]
jazz [('vocal', 92), ('contemporary', 82), ('jazz', 77), ('adult', 63), ('neo', 38)]
metal [('metal', 365), ('alternative', 284), ('nu', 221), ('rock', 218), ('hard', 205)]
pop [('pop', 344), ('indie', 180), ('dance', 164), ('electropop', 129), ('post-teen', 129)]
punk [('pop', 195), ('punk', 138), ('skate', 94), ('alternative', 84), ('melodic', 72)]
rnb [('pop', 219), ('r&b', 213), ('urban', 200), ('dance', 169), ('hip', 136)]
rock [('rock', 638), ('mod

In [8]:
# Find indeces to keep and indeces to remove
t0 = time.time()
cat_genre_dict = {'pop': ['pop'], 
                  'hiphop': ['rap', 'hip', 'trap'],
                  'metal': ['metal'], 
                  'rock': ['classic'], 
                  'jazz': ['jazz'], 
                  'rnb': ['r&b'], 
                  'romance': ['soft'], 
                  'soul': ['soul'], 
                  'indie_alt': ['indie', 'alternative'],
                  'classical': ['classical', 'operatic', 'dance'], 
                  'blues': ['modern', 'blues'],
                  'funk': ['funk', 'disco'], 
                  'punk': ['punk'],
                  'country': ['country']} 
indeces_to_keep = []
indeces_to_remove = []
for index, row in filtered_lyric_SF_df.iterrows():
    keep = False
    category = row['category_name']
    for genre in cat_genre_dict[category]:
        if genre in row['genre_list']:
            keep = True
            break
    if keep == True:
        indeces_to_keep.append(index)
    else:
        indeces_to_remove.append(index)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(len(indeces_to_keep))
len(indeces_to_remove)

Run time: 32.202308177948 seconds
3975


4035

In [9]:
# Create non destructive copy and remove the unnecessary indeces
t0 = time.time()
filtered_lyric_SF_df2 = filtered_lyric_SF_df.drop(indeces_to_remove)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_lyric_SF_df2.head()

Run time: 0.39020800590515137 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,cut,ale,lack,slogan,libido,oop,scorch,muslim,heavyweight,mozambiqu
0,willow,Taylor Swift,pop,8.0,"[dance, pop]",0.392,0.574,7.0,1.0,0.1700,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"[indie, pop, electropop]",0.581,0.584,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,r u ok,Tate McRae,pop,8.0,"[dance, pop, electropop, post-teen]",0.666,0.593,2.0,1.0,0.0373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,BOYSHIT,Madison Beer,pop,8.0,"[dance, pop, electropop, post-teen]",0.638,0.699,1.0,1.0,0.0892,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Safe With Me (with Audrey Mika),Gryffin,pop,8.0,"[dance, pop, tropical, edm]",0.497,0.757,11.0,1.0,0.1270,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Remove the romance punk, and jazz categories
filtered_lyric_SF_df2 = filtered_lyric_SF_df2[filtered_lyric_SF_df2['category_name']!='romance']
filtered_lyric_SF_df2 = filtered_lyric_SF_df2[filtered_lyric_SF_df2['category_name']!='punk']
filtered_lyric_SF_df2 = filtered_lyric_SF_df2[filtered_lyric_SF_df2['category_name']!='blues']
filtered_lyric_SF_df2 = filtered_lyric_SF_df2[filtered_lyric_SF_df2['category_name']!='indie_alt']
len(filtered_lyric_SF_df2)

3069

In [11]:
# Print how many songs there are in each category 
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    song_count = len(filtered_lyric_SF_df2[filtered_lyric_SF_df2['category_name']==cat])
    print(f'There are {song_count} songs in the {cat} category.')

There are 0 songs in the blues category.
There are 33 songs in the classical category.
There are 976 songs in the country category.
There are 162 songs in the funk category.
There are 446 songs in the hiphop category.
There are 0 songs in the indie_alt category.
There are 77 songs in the jazz category.
There are 365 songs in the metal category.
There are 344 songs in the pop category.
There are 0 songs in the punk category.
There are 213 songs in the rnb category.
There are 245 songs in the rock category.
There are 0 songs in the romance category.
There are 208 songs in the soul category.


In [12]:
# Create a list of all genres
t0 = time.time()
genres_list = []
genre_column = list(filtered_lyric_SF_df['genre_list'])
for artist_genres in genre_column:
    genres_list.extend(artist_genres)
genres_list = list(set(genres_list))
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(genres_list)

Run time: 0.009385108947753906 seconds


673

In [13]:
# Find genres that are unique to one category
t0 = time.time()
unique_genres = []
for genre in genres_list:
    genre_df = filtered_lyric_SF_df[pd.DataFrame(filtered_lyric_SF_df['genre_list'].tolist()).isin(np.tile(genre, len(filtered_lyric_SF_df))).any(1).values]
    if len(set(genre_df['category'])) == 1:
        unique_genres.append(genre)
unique_genres = list(set(unique_genres))
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(unique_genres)

KeyError: 'category'

In [609]:
# Update new_genre for any song with a unique genre
#for index, row in song_genres_df.iterrows():
#    for genre in row['filtered_genres']:
#        if (genre in unique_genres) & (row['new_genre']==None):
#            song_genres_df.loc[index, 'new_genre'] = row['category']

In [15]:
# Define the features set
X = filtered_lyric_SF_df2.copy()
X = X.drop(['song_name', 'artist_name', 'category_name', 'category_id', 'genre_list'], axis=1)
X.head()

,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,...,cut,ale,lack,slogan,libido,oop,scorch,muslim,heavyweight,mozambiqu
0,0.392,0.574,7.0,1.0,0.1700,0.8330,0.00179,0.1450,0.529,81.112,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.581,0.584,2.0,1.0,0.2840,0.0805,0.00000,0.3660,0.756,179.954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.666,0.593,2.0,1.0,0.0373,0.3180,0.00000,0.4140,0.329,140.013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.638,0.699,1.0,1.0,0.0892,0.0203,0.00000,0.0879,0.570,121.122,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.497,0.757,11.0,1.0,0.1270,0.0449,0.00000,0.1190,0.507,169.902,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Define the target set
y = filtered_lyric_SF_df2['category_id'].ravel()
y[:5]

array([8., 8., 8., 8., 8.])

In [17]:
# Split into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2301, 9094)
(768, 9094)
(2301,)
(768,)


In [18]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fitting the Model

In [19]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [20]:
# Fit the model
t0 = time.time()
rf_model = rf_model.fit(X_train_scaled, y_train)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

Run time: 28.010973930358887 seconds


# Making Predictions

In [21]:
# Make predictions using the testing data
predictions = rf_model.predict(X_test_scaled)
predictions

array([ 2., 11.,  7.,  2.,  2.,  2.,  2.,  2.,  4.,  2.,  8.,  4.,  2.,
        4.,  2.,  2.,  2.,  2.,  4.,  2.,  4.,  2.,  4.,  7.,  2.,  2.,
        2.,  2.,  7.,  2.,  4.,  2.,  2.,  2.,  8.,  2.,  2.,  2.,  7.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  4.,  8.,  4.,  4.,  4.,  2.,
        7.,  2.,  4.,  2.,  2.,  2.,  4., 10.,  2.,  4.,  2.,  2.,  2.,
        2.,  2.,  2.,  7.,  7.,  7.,  2.,  2.,  2.,  2.,  2.,  2., 13.,
        4.,  2.,  4.,  2., 10.,  7.,  2.,  8.,  2.,  2.,  2.,  8.,  4.,
        2.,  2.,  7.,  2.,  2.,  4.,  4.,  2.,  2.,  2.,  7.,  2.,  4.,
        2.,  2.,  7.,  2.,  8.,  7.,  2.,  4.,  7.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  8.,  2.,  2.,  2.,  4.,  2.,  2.,  7.,  2.,  2.,
        2.,  2.,  4., 10.,  4.,  2.,  4.,  4.,  2.,  4.,  7.,  2.,  2.,
        2.,  4.,  2.,  2.,  4.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  4.,  2.,  2.,  2.,  4.,  2.,  2.,  2.,  2.,
        7.,  2.,  2., 10.,  2.,  2.,  2.,  2.,  7.,  2.,  4.,  7

In [22]:
incorrect = 0
for i in range(len(y_test)):
    if predictions[i] != y_test[i]:
        incorrect+=1
print(len(y_test), incorrect)
(1 - incorrect/len(y_test))*100

768 303


60.546875

# Model Evaluation

In [23]:
# Calculate the accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f'Accuracy Score: {acc_score}')

Accuracy Score: 0.60546875


In [24]:
# Calculate the feature importance in the Random Forest Model
importances = rf_model.feature_importances_
importances

array([1.56648006e-02, 1.66560425e-02, 3.98532821e-03, ...,
       0.00000000e+00, 0.00000000e+00, 4.77871478e-06])

In [25]:
# Sort the features by importance
sorted_features = sorted(zip(importances, X.columns), reverse=True)
sorted_features

[(0.022175874621991845, 'audio_ft_acousticness'),
 (0.01761521534247525, 'audio_ft_speechiness'),
 (0.01665604248388798, 'audio_ft_energy'),
 (0.015664800593269808, 'audio_ft_danceability'),
 (0.014966121796434872, 'audio_ft_duration_ms'),
 (0.013192823896417299, 'audio_ft_instrumentalness'),
 (0.011129907504447993, 'audio_ft_valence'),
 (0.006909434593536065, 'nigga'),
 (0.006747657700777005, 'audio_ft_mode'),
 (0.006527647745044922, 'audio_ft_liveness'),
 (0.006513359048249465, '?'),
 (0.006413113459171381, 'fuck'),
 (0.00627587264940747, 'audio_ft_tempo'),
 (0.006097995643068908, 'like'),
 (0.00593090505978016, 'love'),
 (0.005735084129952821, 'bitch'),
 (0.0055664896950479385, 'know'),
 (0.005501706334799848, 'shit'),
 (0.005369860132720015, 'im'),
 (0.005284995999871667, 'babi'),
 (0.005111339466824511, 'oh'),
 (0.004854678393284802, 'aint'),
 (0.004699126795624839, 'got'),
 (0.004590121522948725, 'get'),
 (0.004585509556472091, 'yeah'),
 (0.004249356561017666, 'dont'),
 (0.004058

# Dropping Lower Ranked Features

In [26]:
# Create a dictionary of features and accuracy scores
features_accuracy = {}
feature_percents = [0.1, 0.2, 0.3, 0.4]
for percent in feature_percents:
    # Define the features and target sets
    important_features = []
    sum = 0
    count = 0
    while sum < percent:
        sum = sum + sorted_features[count][0]
        important_features.append(sorted_features[count][1])
        count+=1
    X = filtered_lyric_SF_df2[important_features]
    y = filtered_lyric_SF_df2['category_id'].ravel()
    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
    # Scale the data
    scaler = StandardScaler()
    X_scaler = scaler.fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    # Fit the random forest model
    rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
    t0 = time.time()
    rf_model = rf_model.fit(X_train_scaled, y_train)
    t1 = time.time()
    print(f'Run time to fit model with {percent*100}% of total features: {t1-t0} seconds')
    # Make predictions
    predictions = rf_model.predict(X_test_scaled)
    acc_score = accuracy_score(y_test, predictions)
    features_accuracy[percent] = acc_score

Run time to fit model with 10.0% of total features: 2.976901054382324 seconds
Run time to fit model with 20.0% of total features: 2.9209837913513184 seconds
Run time to fit model with 30.0% of total features: 3.6202800273895264 seconds
Run time to fit model with 40.0% of total features: 3.8728699684143066 seconds


In [27]:
# Print feature percents and accuracy scores
for percent in features_accuracy:
    acc_score = features_accuracy[percent]
    print(f'Accuracy Score for {percent*100}% of total features: {acc_score}')

Accuracy Score for 10.0% of total features: 0.62890625
Accuracy Score for 20.0% of total features: 0.7005208333333334
Accuracy Score for 30.0% of total features: 0.70703125
Accuracy Score for 40.0% of total features: 0.6979166666666666


In [28]:
# Get the highest accuracy score
best_percent = max(features_accuracy, key=features_accuracy.get)
best_acc = features_accuracy[best_percent]
print(f'The highest accuracy score of {best_acc} was achieved using {best_percent*100}% of all features.')

The highest accuracy score of 0.70703125 was achieved using 30.0% of all features.
